In [76]:
import requests
import pandas as pd
import json

In [81]:
df1 = pd.read_csv('train_data_pajak.csv')
df2 = pd.read_csv('clean_dataset_pajak_header.csv')
dfs = pd.DataFrame([])
def run_curl_command(prompt):
    url = "http://108.137.109.168:11434/api/generate"

    data = {
        "model": "mistral",
        "prompt": prompt,
        "stream": False
    }

    # Sending a POST request
    response = requests.post(url, json=data)

    # Checking the response
    if response.status_code == 200:
        # Return the JSON response
        #return response.json()
        return response.content
    else:
        print(f"Error! Status code: {response.status_code}, Response: {response.text}")
        # Return None or raise an exception, depending on your preference
        return None

In [97]:
def classify_chunk(chunk):
    full_strings = chunk.tolist()
    y_data = '\n'.join(full_strings)
    result = run_curl_command("""
[INST] Please make classification and return JSON formatted answer for the column BARANG in this based on the following criteria:
    - type represents the type of classification ('Service' or 'Product').
    - If the product contains a verb or various activities or offerings that support the production, distribution, marketing, or utilization of palm oil and its derivatives, classify it as 'service'.
    - If the product is raw material or a product typically refers to the tangible goods derived from palm oil classify it as 'product'.
    - confidenceLevel represents the confidence level of the classification.
    - explanation provides additional information about the classification.
    - Only return the JSON formatted answer and NOTHING else.
    Example of JSON Formatted Answer:
[/INST]
        {{"productName":"Analisa  Pengujian INSPEKSI MINYAK KELAPA SAWIT DAN PRODUK TURUNANNYA dan Analisa  Pengujian 1 Sa","type":"Service","confidenceLevel":0.9,"explanation":"This is service for testing analysis inpection"}},
        {{"productName":"TANDAN BUAH SEGAR KELAPA SAWIT TAHUN TANAM 1999 1700094 KG","type":"Product","confidenceLevel":0.9,"explanation":"This is a product of Fresh Fruit Oil Palm"}},
        {{"productName": "BLANKO SOUNDING PALM KERNEL","type": "Service","confidenceLevel": 0.9,"explanation": "This is a sounding service for palm kernel"}}
[INST]
<<<
    This is the text to classify:
    {y}
>>>
- Only return JSON Formatted answer and nothing else
- Check and correct the JSON data by ensuring that all property names are enclosed in double quotes.
- The trailing comma at the end should be removed since it's the last element in the JSON object.
- Ensure that the strings are properly terminated. Dont let any comma on the end of key or value
- You need to wrap all JSON objects within square brackets "[" and "]" to form a valid JSON array.
- Make sure there's no extra comma after the last value in each dictionary, which is causing the error.
[/INST]
    """.format(y=y_data))['response']
    return result

In [83]:
results = []
for i in range(0, 51,5):
    chunk = df2['NamaBarang'].iloc[i:i+5]
    result = classify_chunk(chunk)
    if result:
        results.append(result)

In [84]:
for result in results:
    try:
        jsonData = json.loads(result)
        if isinstance(jsonData, list):
            for json_obj in jsonData:
                newDfs = pd.DataFrame([json_obj])  # Convert dictionary to DataFrame
                dfs = pd.concat([dfs, newDfs], ignore_index=True)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        print(f"Problematic JSON: {result}")
    except Exception as e:
        print(f"Error processing JSON: {e}")
        print(f"Problematic JSON: {result}")

Error decoding JSON: Expecting property name enclosed in double quotes: line 2 column 5 (char 7)
Problematic JSON:  {
    ["{
        \"productName\":\"KALIBRASI SOUNDING TAPE CPO\",
        \"type\":\"Service\",
        \"confidenceLevel\":0.9,
        \"explanation\":\"This is a service for calibrating sounding tape for CPO\"
    }",
    {
        "\"productName\":\"Loading Survey of CPKO 1999887 MT and CPO 2503809 MT at Dumai on 1516102021\",
        \"type\":\"Service\",
        \"confidenceLevel\":0.8,
        \"explanation\":\"This is a service for loading survey of CPKO and CPO at Dumai on specific dates\"
    },
    {
        "\"productName\":\"Loading Survey of CPO 2999998 MT at Dumai on 0607122021\",
        \"type\":\"Service\",
        \"confidenceLevel\":0.8,
        \"explanation\":\"This is a service for loading survey of CPO at Dumai on specific date\"
    },
    {
        "\"productName\":\"Loading Survey of CPO 3000301 MT at Tayan on 1920102021\",
        \"type\":\"S

In [95]:
for result in results:
    try:
        # Remove leading and trailing whitespaces
        result = result.strip()
        result = result.strip('[]')
       # result = result.strip('""', '')

        # Remove backslashes
        result = result.replace('\\', '')
        
         # Remove the extra square brackets
        result = result.strip('[').strip(']')
        
      # Load JSON data directly from the array
        jsonData = json.loads(result)
        
       # result = result.dump(result, separators=(',', ':'))
        
        # Check if jsonData is a list of dictionaries
        if isinstance(jsonData, list):
            for json_obj in jsonData:
                newDfs = pd.DataFrame([json_obj])  # Convert dictionary to DataFrame
                dfs = pd.concat([dfs, newDfs], ignore_index=True)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        print(f"Problematic JSON: {result}")
    except Exception as e:
        print(f"Error processing JSON: {e}")
        print(f"Problematic JSON: {result}")

Error decoding JSON: Extra data: line 5 column 10 (char 227)
Problematic JSON:     {        "productName": "ANALISA SAMPEL RBD PALM STEARIN",
            "type": "Service",
            "confidenceLevel": 0.9,
            "explanation": "This is a service for testing analysis of RBD Palm Stearin"
        },
        {
            "productName": "Palm Kernel Expeller PKE",
            "type": "Product",
            "confidenceLevel": 0.9,
            "explanation": "This is a product of Palm Kernel Expeller"
        },
        {
            "productName": "Jasa Uji Laboratorium KJU No XX XX",
            "type": "Service",
            "confidenceLevel": 0.9,
            "explanation": "This is a laboratory testing service"
        }
    
Error decoding JSON: Expecting property name enclosed in double quotes: line 2 column 5 (char 6)
Problematic JSON: {
    ["{
        "productName":"KALIBRASI SOUNDING TAPE CPO",
        "type":"Service",
        "confidenceLevel":0.9,
        "explanation

In [65]:
display(dfs)

""


In [143]:
print(result)

 [
    {
        "productName": "CRUDE PALM OIL",
        "type": "Product",
        "confidenceLevel": 0.95,
        "explanation": "This is a product of crude palm oil."
    },
    {
        "productName": "CRUDE PALM OIL FFA 5 max MI 05 max",
        "type": "Product",
        "confidenceLevel": 0.9,
        "explanation": "This is a product of Crude Palm Oil with FFA (Free Fatty Acid) content within the specified limit."
    },
    {
        "productName": "7100 KG",
        "type": "Service",
        "confidenceLevel": 0.8,
        "explanation": "This is a service for sale of 7100 kg of Crude Palm Oil."
    },
    {
        "productName": "PK",
        "type": "Product",
        "confidenceLevel": 0.95,
        "explanation": "Palm Kernel is a product derived from the seed of palm oil plant"
    },
    {
        "productName": "PALM KERNEL INTI SAWIT",
        "type": "Product",
        "confidenceLevel": 0.98,
        "explanation": "Palm Kernel Inti (Fresh Palm Kernel) is a pro